In [ ]:
## DATASET PER LE MOSSE
os.chdir("/Users/tommasodimario/Documents/GitHub/statistics-101/project/src")

vialibera = False

gamecount = 0
count = 0   
partita = ""
with open("mosse.csv", "w") as file:
    writer = csv.writer(file)
    # TR: Time Remaining
    # T: Time in seconds from the beginning of the match
    # HRDelta: difference of the heart rate at the moment of the move
    # TPR: Time per move
    
    columns = ["MoveS","MovePGN", "Player", "TimeUsed", "Time", "Evaluation","HRDelta", "TPM", "Board", "Game", "DeltaPos", "AbsHRAfter","AbsHRBefore"]
    # move ok, board ok, player ok, tr ok, t ok, hrdelta ok, manca tpr
    writer.writerow(columns)
    file.close()

for root,dirs, files in os.walk("games"):
    if str(root)  != "games":
        partita = f"Game{root[-6:]}"
        gamecount += 1
        
    df = pd.read_csv("partite.csv")

    df = df.pivot_table(values='Value',
                            index='Time',
                            columns=['Game', 'Player', 'Statistics'])
    for file in files:
        #qui siamo dentro i file di una singola partita
        filename = os.fsdecode(file)
        
        #print(filename)
        
        #leggiamo i battiti
        
        if filename.endswith(".pgn"):
            vialibera = True
            filename = filename[:-3] + "csv"
            # print(f"processando {filename}")
            
        
            moves = get_moves(f"{str(root)}/{filename}")
 
            last_pos = 0.0
            lastbb = 0
            lastbw = 0
            added = 0
            ret = []
            # print(moves)
            with open(f"{str(root)}/{filename}", newline='') as f:
                pgn = []
                time_last_move_w = 0.0
                time_last_move_b = 0.0
                
                for row in csv.reader(f):
                    if row[0] == "Time":
                        continue
                    
                    diffw = 1
                    diffb = 1
                    move = eval(row[3])
                    abshrdopo = 0
                    abshrprima = 0
                    if move != []:
                        trem = eval(row[2])
                        # aggiungi max_delta(5 secondi da quella mossa)
                        # aggiungi differenza di posizione di mossa in mossa
                        # togli mosse che clashano
                        for _ in move:
                            if added % 2 == 0:
                                if "#" not in row[1]:
                                    delta_pos = float(row[1]) - last_pos
                                    last_pos = float(row[1])
                                else:
                                    if "-" in row[1]:
                                        delta_pos = -20 - last_pos
                                        last_pos = -20
                                    elif "+" in row[1]:
                                        delta_pos = 20 - last_pos
                                        last_pos = 20
                                try:
                                    battw = df[partita]["w"]["HR"][int(row[0])]
                                    if math.isnan(battw):
                                        battw = lastbw
                                    lastbw = battw
                                    for i in range(5):
                                        ## ABSHR per 5 secondi dopo
                                        try: 
                                            print(partita)
                                            nbw = int(df[partita]["w"]["HR"][int(row[0]) + i])
                                            
                                        except:
                                            pass
                                        else:
                                            nb = int(df[partita]["w"]["HR"][int(row[0]) + i])
                                            ab = abs(abs(int(battw)) + abs(int(nbw)))
                                            abshrdopo = max( abshrdopo, ab)
                                        
                                        ## ABSHR per 5 secondi prima
                                        if int(row[0]) - i >= 0:
                                            nb = int(df[partita]["w"]["HR"][int(row[0]) - i])
                                            ab = abs(abs(int(battw)) + abs(int(nb)))
                                            abshrprima = max( abshrprima, ab)
                                        else:
                                            abshrprima = 0
                                    
                                    
                                except:
                                    battw = lastbw
                                    
                                
                                ret.append(moves[added] + [move,"w",trem[0],int(row[0]), row[1], battw, trem[0] - time_last_move_w, row[-1], partita,  min(abs(delta_pos), 20),abshrdopo,abshrprima]) 
                                time_last_move_w = trem[0]
                                
                            else:
                                if "#" not in row[1]:
                                    delta_pos = last_pos - float(row[1])
                                    last_pos = float(row[1])
                                    
                                else:
                                    if "-" in row[1]:
                                        delta_pos = -20 - last_pos
                                        last_pos = -20
                                    elif "+" in row[1]:
                                        delta_pos = 20 - last_pos
                                try:
                                    battb = df[partita]["b"]["HR"][int(row[0])]
                                    if math.isnan(battb):
                                        battb = lastbb
                                    lastbb = battb
                                    for i in range(5):
                                        
                                        try: 
                                            nbb = int(df[partita]["b"]["HR"][int(row[0]) + i])
                                            
                                        except:
                                            pass
                                        else:
                                            nbb = int(df[partita]["b"]["HR"][int(row[0]) + i])
                                            ab = abs(abs(int(battb)) + abs(int(nbb)))
                                            abshrdopo = max( abshrdopo, ab)
                                            
                                            
                                        
                                        if int(row[0]) - i > 0:
                                            nb = int(df[partita]["b"]["HR"][int(row[0]) - i])
                                            ab = abs(abs(int(battb)) + abs(int(nb)))
                                            abshrprima = max( abshrprima, ab)
                                        else:
                                            abshrprima = 0
                                         
                                        
                                except:
                                    battb = lastbb
                                    
                                
                                
                                ret.append(moves[added] + [move,"b",trem[1],int(row[0]), row[1], battb, trem[1] - time_last_move_b, row[-1], partita, min(abs(delta_pos), 20), abshrdopo,abshrprima])
                                time_last_move_b = trem[1]
                                
                            added += 1
                        

            if vialibera:            

                vialibera =False
                with open("mosse.csv", "a") as file:
                    writer = csv.writer(file)
                    
                    for move in ret:
                        writer.writerow(move)
                                    
                                   



In [ ]:
# Create a dataset with only the two variables 
dff = df[["Time","AbsHRBefore", "AbsHRAfter", "DeltaPos", "HRDelta", "Game"]]


In [ ]:
conflict_time = 0
conflict_set = set()
for i, row in dff.iterrows():
    for j, row2 in dff.iloc[i+1:i+11].iterrows():
        print(row2[0])
        if int(row2[0]) - int(row[0]) <= 10:
            print(row2[0],row[0])
            conflict_set.add(i)
            conflict_set.add(j)
            
        
            
        
            

In [ ]:
for el in list(conflict_set):
    dff = dff.drop(el)

In [ ]:
plt.scatter(dff["DeltaPos"], dff["AbsHRAfter"])

In [ ]:
plt.scatter(dff["AbsHRBefore"], dff["AbsHRAfter"])

In [ ]:
## BLUNDERS o OTTIME MOSSE
ndff =  dff[["AbsHRBefore","AbsHRAfter", "MoveS", "Time", "DeltaPos", "MoveS", "Board"]]

for row in ndff.iterrows():
    if row[1][4] > 3.0:
        print(row[1][-2])
        print(row[1]["AbsHRBefore"],row[1]["AbsHRAfter"])
        display(chess.Board(row[1][-1]))

: 